# What Can `execnb` do?

This notebook is a [SolveIt](https://solveit.fast.ai/)-style exploration of [https://github.com/AnswerDotAI/execnb/](https://github.com/AnswerDotAI/execnb/). Here I am following the SolveIt process in a Jupyter notebook to learn new things.

## Understand the Problem

I've been interested in learning what the open-source `execnb` package does and how it works. I don't have a particular use case, other than wanting to know more so I can contribute to it or to code that uses it.

## Devise a Plan

* Study the examples defined in the `execnb` notebooks
* Create my own examples patterning them after those examples
* Use literate programming in this notebook to write about my findings

## Carry Out the Plan

Let's see what execnb can do!

In [60]:
from execnb.nbio import *
from execnb.shell import *
from fastcore.all import *
from fasthtml.common import *
from IPython.display import HTML
from pathlib import Path

## Setup: Let's Grab Some Jupyter Notebooks

We have notebooks in `nbs/` in this repo which we can look at.

In [10]:
root = Path('../nbs').parent
nb_dir = root/'nbs'
nb_dir

Path('../nbs')

Yay for fastcore `L` lists and chainable operations like `sorted`:

In [38]:
nbs = L(nb_dir.glob('*.ipynb')).sorted()
nbs

(#13) [Path('../nbs/2023-07-29-nbdev.ipynb'),Path('../nbs/2024-07-14_SemanticUI_Cards.ipynb'),Path('../nbs/2024-07-15-Printing_Components.ipynb'),Path('../nbs/2024-07-16_Xtend_Pico.ipynb'),Path('../nbs/2024-07-29-Auth.ipynb'),Path('../nbs/2024-07-29-Delegates-Decorator.ipynb'),Path('../nbs/2024-07-29-FH-by-Example.ipynb'),Path('../nbs/2024-08-04-Claudette.ipynb'),Path('../nbs/2024-08-05-Claudette-FastHTML.ipynb'),Path('../nbs/2024-12-23-Daddys_Snowman_Card.ipynb'),Path('../nbs/2024-12-23-print.ipynb'),Path('../nbs/2024-12-23-read_nb_and_render_nb.ipynb'),Path('../nbs/2024-12-24-execnb.ipynb')]

## Reading a Jupyter Notebook with `read_nb`

`read_nb` comes from `execnb.nbio`:

In [18]:
nb = read_nb(nbs[9])
L(nb['cells'])[1]

```json
{ 'cell_type': 'markdown',
  'idx_': 1,
  'metadata': {},
  'source': "Here we are checking the numbers from our daughter's snowman card "
            'to Daddy. She gave him math problems to solve and a snowman joke.'}
```

That's nice that we can get any cell, and get its info!

## Jupyter Notebook Cells

Okay, let's grab the source of cells:

In [39]:
def get_source(cell): return cell['source']

In [21]:
cells = L(nb['cells']).map(get_source)
cells

(#6) ["# Daddy's Snowman Card","Here we are checking the numbers from our daughter's snowman card to Daddy. She gave him math problems to solve and a snowman joke.",'200000','**AI Prompt**\n\nWhat is 200000 (2 followed by 5 zeroes)?',"**AI Response**\n\nThat's two hundred thousand.",'100+200000']

Let's see if those are AttrDicts:

In [29]:
nb.cells[0]

```json
{ 'cell_type': 'markdown',
  'idx_': 0,
  'metadata': {},
  'source': "# Daddy's Snowman Card"}
```

Yes!

Let's use this nice AttrDict to get the source of a cell:

In [31]:
L(nb.cells)[1].source

"Here we are checking the numbers from our daughter's snowman card to Daddy. She gave him math problems to solve and a snowman joke."

In [30]:
def get_source(cell): return cell.source
cells = L(nb.cells).map(get_source)
cells

(#6) ["# Daddy's Snowman Card","Here we are checking the numbers from our daughter's snowman card to Daddy. She gave him math problems to solve and a snowman joke.",'200000','**AI Prompt**\n\nWhat is 200000 (2 followed by 5 zeroes)?',"**AI Response**\n\nThat's two hundred thousand.",'100+200000']

Yes, did it!

## Jupyter Notebook Metadata

Sooo...besides cells there's metadata, right?

In [33]:
nb.metadata

```json
{ 'kernelspec': { 'display_name': '.venv',
                  'language': 'python',
                  'name': 'python3'},
  'language_info': { 'codemirror_mode': {'name': 'ipython', 'version': 3},
                     'file_extension': '.py',
                     'mimetype': 'text/x-python',
                     'name': 'python',
                     'nbconvert_exporter': 'python',
                     'pygments_lexer': 'ipython3',
                     'version': '3.12.7'}}
```

Seems useful. Might be worth printing the Python version at least:

In [36]:
nb.metadata.language_info.version

'3.12.7'

I feel like I'd want to print the Python version for every notebook I'm publishing.

The version of each imported package would be nice too. Looks like that's beyond the scope of execnb most likely. Or is it?

## CaptureShell

Looks like we can run a Jupyter notebook cell:

In [41]:
s = CaptureShell(mpl_format='retina')
s

In [43]:
s.run_cell('print("hi")')

```json
{ 'display_objects': [],
  'exception': None,
  'quiet': False,
  'result': result: None; err: None; info: <cell: print("hi"); id: None>,
  'stderr': '',
  'stdout': 'hi\n'}
```

Printing didn't have a result. How about a Python expression:

In [44]:
s.run_cell('1+1+1')

```json
{ 'display_objects': [],
  'exception': None,
  'quiet': False,
  'result': result: 3; err: None; info: <cell: 1+1+1; id: None>,
  'stderr': '',
  'stdout': ''}
```

Ah, so we can see the result of evaluating it.

What about a Markdown cell?

In [47]:
s.run_cell('# Hi')

```json
{ 'display_objects': [],
  'exception': None,
  'quiet': False,
  'result': result: None; err: None; info: <cell: # Hi; id: None>,
  'stderr': '',
  'stdout': ''}
```

In [49]:
s.run('# Hi')

[]

Looking at `execnb.shell`, I think it's just for Python code right now.

In [48]:
s.run("print(1)")

[{'name': 'stdout', 'output_type': 'stream', 'text': ['1\n']}]

## Rendering cell outputs

I'm offline and can't install dependencies, but I can see that `render_outputs` can render some outputs of executed cells like matplotlib plots.

What about a simple Python expression?

In [51]:
o = s.run("1+2+3")
o

[{'data': {'text/plain': ['6']},
  'metadata': {},
  'output_type': 'execute_result',
  'execution_count': None}]

In [52]:
render_outputs(o)

'<pre ><code>6</code></pre>'

What about some FastHTML?

In [55]:
o = s.run("""from fasthtml.common import *
P("Hi")""")
o

[{'data': {'text/plain': ["p(('Hi',),{})"],
   'text/markdown': ['```html\n', '<p>Hi</p>\n', '\n', '```']},
  'metadata': {},
  'output_type': 'execute_result',
  'execution_count': None}]

In [56]:
render_outputs(o)

'<pre><code class="language-html">&lt;p&gt;Hi&lt;/p&gt;\n\n</code></pre>\n'

The `HTML` function from `IPython.display` looks useful here:

In [61]:
HTML(render_outputs(o))

## Completions

`SmartCompleter` extends `IPCompleter` from `IPython.core.completer`. We can try instantiating one and seeing what it does:

In [62]:
cc = SmartCompleter(get_ipython())
cc

In [65]:
cc("pr")

['print', 'properties', 'property']

This is quite interesting!

I'm currently offline and feel like I need to read the IPython docs and source to understand more.

## Reflect

In [ ]:
I've studied the 2 notebooks in `execnb`: `nbio` and `shell`

I created examples to explore:
* `read_nb`'s returned nb cells and metadata
* `CaptureShell` and its `run_cell()` and `run()` methods
* `render_outputs()` and `HTML()`
* `SmartComplete